In [63]:

from variables import sourcepath
import os
import pandas as pd
import numpy as np
import etl.dim_author as auth
import etl.dim_journal as jour
import etl.common_functions as cof
import etl.dim_paper as pape
import etl.database as db
from db_credentials import dwh_db_connection_params
import roman
import datetime


In [56]:
eng, psycop2connect=db.initialize_engine(connection_params=dwh_db_connection_params)

In [58]:
articles_df, references_df=pape.extract_all_papers()
#first prepare papers from 'papers_final'
articles_prep=pape.transform_papers(articles_df, eng)
references_prep=pape.transform_references(references_df, eng)
final_source_papers=pape.merge_all_papers(references_prep, articles_prep)
papers_in_dwh=db.load_full_table(eng, 'dim_paper')


/home/luisa_waack/virtualenvs/thesis3.9/lib/python3.9/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [65]:
delta_papers, delta_keywordgroup, delta_keywordbridge, delta_authorgroup, delta_authorbridge=pape.find_delta_papers(final_source_papers, papers_in_dwh)

ValueError: too many values to unpack (expected 5)

Option 1: in paper transformation alle funktionen implemntieren, die author-, keyword- und journal transformation auch haben und hier noch mal alles mit den article_ids ausführen um die foreign keys in der db nsuchen zu können (lookup)

Option 2: author-, keyword- und journal transformation sind class attributes von paper transformation, und beinhalten jeweils noch eine Funktion, welche den Link zu paper_id bzw citekey herstellt und ausgibt.

Option 3: einen anderen algorithmus für primary keys nutzen, der auf basis eines strings immer die gleiche nummer zurückgibt (hash value, e.g. SHA1), dann ist kein lookup nötig. Vorteil: Independent / parallel processing, child table kann vor parent table geladen werden. Kombiniert mit 1 oder 2. 

DEPENDENCY INJECTION!?
https://roelantvos.com/blog/using-a-natural-business-key-the-end-of-hash-keys/

In [9]:
print(ref.shape)
print(pap.shape)

(192003, 5)
(7479, 5)


In [5]:
all.dropna(axis=0, how='all', inplace=True)
all.shape

(174318, 5)

In [7]:
#all.fillna({'journal': 'MISSING', 'volume':0, 'issue': 0, 'publisher': 'MISSING', 'place': 'MISSING'}, inplace=True)